<a href="https://colab.research.google.com/github/hardik-vala/misc/blob/main/cofounder_hunt_retro_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis the 'Candidates' sheet from my [Co-founder CRM](https://docs.google.com/spreadsheets/d/1siplh_pCkfQCVJOtaWtQiiBIq9otgqm6RPaMu6jnN9c/edit?pli=1&gid=0#gid=0), as of Aug 30, 2024.

> WARNING: Data contains inaccuracies when it comes to reporting the loss / rejection reasons, only one reason is logged (the most influential) even though many may apply.

In [1]:
!pip install plotly

In [2]:
import pandas as pd

df = pd.read_csv('/content/Co-founder CRM - Candidates.csv')
df.columns

Index(['Date Added', 'Status', 'Reason', 'Name', 'Contact info.', 'Source',
       'Inbound vs Outbound', 'Stage', 'Notes'],
      dtype='object')

## Number of candidates,

In [3]:
df.shape[0]

207

## Breakdown by candidate source,



In [4]:
import plotly.graph_objects as go

colors = {'YC': 'orange', 'LI': 'blue', 'Personal': 'gray', 'AI Tinkerers': 'purple', 'Xoogler': 'yellow'}

labels = df['Source'].value_counts().keys()
values = df['Source'].value_counts().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent+value')])
fig.update_traces(marker=dict(colors=[colors[v] for v in labels]))
fig.show()

## Breakdown by pipeline status,

In [5]:
labels = df['Status'].value_counts().keys()
values = df['Status'].value_counts().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent+value')])
fig.show()

## Funnel

In [6]:
import plotly.express as px

df_filtered = df[df['Stage'] != 'To Contact']
df_filtered = df_filtered[df_filtered['Stage'] != 'Rejected']

stages = ["Contacted", "Responded", "Screening Call Scheduled", "Deep-Dive", "Trial"]
temp = df_filtered.groupby('Stage').size().reset_index(name='counts')
temp = temp.sort_values(by=['Stage'], key=lambda x: x.map({v: -i for i, v in enumerate(stages)}))
temp['counts'] = temp['counts'].cumsum()
temp = temp.sort_values(by=['counts'], ascending=False)
fig = px.funnel(temp, x='Stage', y='counts', category_orders={"Stage": stages})
fig.show()

In [7]:
temp['conversion'] = (temp['counts'] / temp['counts'].iloc[0]) * 100
temp

,Stage,counts,conversion
0,Contacted,195,100.000000
2,Responded,104,53.333333
3,Screening Call Scheduled,83,42.564103
1,Deep-Dive,20,10.256410
4,Trial,4,2.051282


### Funnels for each candidate source

In [8]:
for source in df_filtered['Source'].unique():
  temp = df_filtered[df_filtered['Source'] == source]
  temp = temp.groupby('Stage').size().reset_index(name='counts')
  temp = temp.sort_values(by=['Stage'], key=lambda x: x.map({v: -i for i, v in enumerate(stages)}))
  temp['counts'] = temp['counts'].cumsum()
  temp = temp.sort_values(by=['counts'], ascending=False)
  fig = px.funnel(temp, x='Stage', y='counts', title=source, category_orders={"Stage": stages})
  fig.show()

  temp['conversion'] = (temp['counts'] / temp['counts'].iloc[0]) * 100
  print(temp)

                      Stage  counts  conversion
0                 Contacted      12  100.000000
2  Screening Call Scheduled      11   91.666667
1                 Deep-Dive       4   33.333333
3                     Trial       2   16.666667


                      Stage  counts  conversion
0  Screening Call Scheduled       5       100.0
1                     Trial       1        20.0


                      Stage  counts  conversion
0                 Contacted      43  100.000000
1                 Responded      13   30.232558
2  Screening Call Scheduled       8   18.604651


                      Stage  counts  conversion
0                 Contacted     126  100.000000
2                 Responded      70   55.555556
3  Screening Call Scheduled      57   45.238095
1                 Deep-Dive      15   11.904762
4                     Trial       1    0.793651


                      Stage  counts  conversion
0                 Contacted       9  100.000000
1                 Responded       5   55.555556
2  Screening Call Scheduled       2   22.222222


## Reasons

In [9]:
for status in ['Lost', 'Rejected']:
  df_filtered = df[df['Status'] == status]

  labels = df_filtered['Reason'].value_counts().keys()
  values = df_filtered['Reason'].value_counts().values

  fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent+value')])
  fig.update_layout(title_text=status)
  fig.show()

### Reasons for each candidate source,

In [10]:
for source in df['Source'].unique():
  df_filtered = df[df['Source'] == source]

  labels = df_filtered['Reason'].value_counts().keys()
  values = df_filtered['Reason'].value_counts().values

  fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent+value')])
  fig.update_layout(title_text=source)
  fig.show()

### Reasons for each stage,

In [14]:
stages = ["Responded", "Screening Call Scheduled", "Deep-Dive", "Trial"]

reasons = df['Reason'].unique()

fig = go.Figure()

for reason in reasons:
  df_filtered = df[df['Reason'] == reason]
  stage_counts = df_filtered['Stage'].value_counts()

  fig.add_trace(go.Bar(
      name=reason,
      x=stages,
      y=[stage_counts.get(stage, 0) for stage in stages]
  ))

fig.update_layout(title_text='Reasons by Stage', barmode='stack', xaxis_title="Stage", yaxis_title="Count")
fig.show()

## Takeaways

* 2% overall trial conversion rate, with 10% making it past screening
* Personal network offers the best trial conversion (~17%)
* Got one trial from Xoogler community, and one from the YC platform (but 1/126, < 1%)
* LI and other sources are terrible - Booked intro appointments but nobody advanced beyond
* People on LI working at "Stealth Startup" are not necessarily looking for a co-founder (majority already had advanced pretty far with an idea or already had a co-founder)
  * I'm not interested in joining an existing team with a mature idea as a late co-founder
* Usually doesn't work out because either interests don't line up, the candidate is too inexperienced, or there's a chemistry issue
  * I might have been too harsh in dismissing candidates because of perceived inexperience...but I'm willing to entertain these prospects if I can quickly assess their ability to learn fast, error-correct, and integrate feedback. If I believe the person isn't intellectually humble, then its a hard pass.
* I was aiming to nurture 10 active and qualified leads towards the end of August, but I have 4, which suggests I should've been more lenient during qualification, or accomplished more outbound.
* I have designated 14 people on standby ("Later"), which is roughly the number of leads that passed the screening test, which I believe are high-quality, but simply cannot commit because of timing (eg. immigration, soul searching). So a big reason good leads aren't advancing is because of timing issues. (Next time, capture the reason.)

## Open Questions

* Was wholly committing to the co-founder search for one month worth it?
* The idea of a funnel is kinda interesting because usually in a sales or product context, you want to push more people further down the funnel. But here, you want to end up with one person at the end (hopefully the best fit).
* Inbound vs. Outbound?
